# Evaluation Result

In [1]:
from sklearn import metrics
from pylib.tlsh_lib import *
import numpy as np
import pandas as pd
import time
tic = time.perf_counter() # experiment time counter 

In [2]:
#Remove NaN function
nan = False
def containNan(labelList):
    for lable in labelList:
        if lable == 'n/a':
            nan = True
            return nan
        
def removeNan(hashlist, labelList):
    count = -1
    newhashlist = []
    newlabelList = []
    
    for lable in labelList:
        count += 1
        if lable != 'n/a':
            newhashlist.append(hashlist[count])
            newlabelList.append(labelList[count])
    return newhashlist, newlabelList

In [3]:
datafile = "dataDir2/mb_1K.csv" #<----------Change this file size

(path,file) = datafile.split("/") #save file path
(filename,filetype) = file.split(".") #save file type

(tlist, [labelList, dateList, hashList]) = tlsh_csvfile(datafile) # return (tlist, [labelList, dateList, hashList])
#(tlist, labelList) = tlsh_csvfile(datafile) 

#remove Nan Value
#(tlist, labelList) = removeNan(tlist, labelList)

print("Number of samples is " + str(len(tlist)))
print("Number of Unique Lable is " + str(len(set(labelList))))
print(hashList[0:10])
nlable = len(set(labelList))
nan = containNan(labelList)

Number of samples is 1000
Number of Unique Lable is 56
['12288:TcinWHCM2K4C2fut2I2U9WLDQ9RnX/lmSG02h8W62NtYy:TR3CBtNTpWNT6y', '12288:3b1f+WVBULxM6IdO6r1uH38W9D+4ncY2BdcvwGqIg3/FTj32c4zWKO:rB+s8eOOI38z4cXSwKg3p7Vx', '12288:lH1rtcf9uFvwrp+8uGqARQi1+3lwbcSpsPP7r9r/+ppppppppppppppppppppppJ:VzF8oQXSi1Oevw1q', '12288:0jdRStsZZzk8/dRStsZZzkldRStsZZzkbgoPYj3YnSn93kEcnVAcx6lQr1Mjg9W2:NCdktCdkiCdkbkjInJmcIlQGjZCRClk', '12288:sULkUtcX1J7ygKXcNxa64qnYDNWXpcJyQPRI8rv:sUL9tcFJ7ygAcUWsyiRI87', '768:dLt2gJbT/5wQu2rNnFfIr8t7sJmn3P5DBlyF8yPyeJ8YH+Rs+FWYQw+6VzBTPv7r:L3JbTBwQuA5Fm86+dBlyFzyeJ8U+FWYj', '12288:dYWWHCM2K4CUY4lbFe5h5LtEjthcgbdQtAa+xaan:J3CUH9KLOTFtxa+', '12288:vjBID6xsmQECVwPvjY/P5vaDx9r4A7Pm6r0c8Ayfk4MwzqEj:a5nGahKx9lrVr0nAT10j', '12288:9ULkUtcX1J7ygKXcNxa64qnYDNWXpcJyQPRI8rA:9UL9tcFJ7ygAcUWsyiRI8s', '24576:G+oELLuVzvLM223WfM8oFJYEVOdsCDd3lF4iDWz3lbWaDTvT:hoEfkzDM2sL862tDd3AQWz1blPr']


In [4]:
df = pd.DataFrame()

def getResult(fh,c,labelList, clusterNumber):
    d = {word: key for key, word in enumerate(set(labelList))}
    labelList_id = [d[word] for word in labelList]
    a = np.array(labelList_id).reshape(-1, 1)
    
    #decimal place
    dp = 4
    
    homo = round(metrics.homogeneity_score(labelList_id, clusterNumber),dp)
    silh = round(metrics.silhouette_score(a, clusterNumber, metric='euclidean'),dp)
    cali = round(metrics.calinski_harabasz_score(a, clusterNumber),dp)
    dav = round(metrics.davies_bouldin_score(a, clusterNumber),dp)
    
    print("Homogeneity score is " + str(homo))
    print("Silhouette score is " + str(silh))
    print("Calinski harabasz score is " + str(cali))
    print("Davies bouldin score is " + str(dav))
    
    result = {"File": str(filename),
              "nSample": int(len(tlist)),
              "Hash": str(fh),
              "Cluster": str(c),
              "Has_n/a": bool(nan),
              "nLabel": int(nlable),
              "nCluster": int(max(clusterNumber)),
              "Time(s)": float(end),
              "Homo.":float(homo),
              "Sil.":float(silh),
              "Cal.":float(cali),
              "Dav.":float(dav)}
    return result

# TLSH Agglomerative Clustering

In [5]:
start = time.perf_counter()

clusterNumber = assignCluster(tlist, nlable)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_hac_out.txt"
#outputClusters(outfile, tlist, clusterNumber, labelList, quiet=True)

dict = getResult("tlsh","hac",labelList,clusterNumber)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.6222 seconds
Homogeneity score is 0.4868
Silhouette score is -0.5495
Calinski harabasz score is 10.0111
Davies bouldin score is 84.7388


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.6222,0.4868,-0.5495,10.0111,84.7388


# TLSH HAC-T

In [6]:
from pylib.hac_lib  import *

hac_resetDistCalc()
start = time.perf_counter()

res = HAC_T(datafile, CDist=30, step3=0, outfname="tmp.txt", cenfname="tmp2.txt") 

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_hac-t_out.txt"
#outputClusters(outfile, tlist, res, labelList, quiet=True)

nclusters = max(res)
nDistCalc = hac_lookupDistCalc()

print("Number of cluster is " + str(nclusters))
print("Number of Distance Calculated is " + str(nDistCalc))

dict = getResult("tlsh","hac-t",labelList,res)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.2077 seconds
Number of cluster is 85
Number of Distance Calculated is 227245
Homogeneity score is 0.3453
Silhouette score is -0.7152
Calinski harabasz score is 3.6595
Davies bouldin score is 55.2025


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.6222,0.4868,-0.5495,10.0111,84.7388
1,mb_1K,1000.0,tlsh,hac-t,1.0,56.0,85.0,0.2077,0.3453,-0.7152,3.6595,55.2025


# TLSH DBSCAN

In [7]:
resetDistCalc()
start = time.perf_counter()

res = runDBSCAN(tlist, eps=30, min_samples=2, algorithm='auto')

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_dbscan_out.txt"
#outputClusters(outfile, tlist, res.labels_, labelList, quiet=True)

nclusters = max(res.labels_)
nDistCalc = lookupDistCalc()

print("nclusters is " + str(nclusters))
print("nDistCalc is " + str(nDistCalc))

dict = getResult("tlsh","dbscan",labelList,res.labels_)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.975 seconds
nclusters is 71
nDistCalc is 961443
Homogeneity score is 0.3336
Silhouette score is -0.7217
Calinski harabasz score is 4.3249
Davies bouldin score is 39.4504


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.6222,0.4868,-0.5495,10.0111,84.7388
1,mb_1K,1000.0,tlsh,hac-t,1.0,56.0,85.0,0.2077,0.3453,-0.7152,3.6595,55.2025
2,mb_1K,1000.0,tlsh,dbscan,1.0,56.0,71.0,0.9750,0.3336,-0.7217,4.3249,39.4504


# TLSH KMeans

In [8]:
from sklearn.cluster import KMeans

def runKMean(tlist, n):
    tdata = tlist2cdata(tlist)
    res = KMeans(n_clusters=n, random_state=0).fit(tdata)
    return(res)

In [9]:
start = time.perf_counter()

res = runKMean(tlist, nlable)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_kmean_out.txt"
#outputClusters(outfile, tlist, res.labels_, labelList)

dict = getResult("tlsh","kmeans",labelList,res.labels_)
df = df.append(dict, ignore_index = True)
df

Code ran in 0.1795 seconds
Homogeneity score is 0.4641
Silhouette score is -0.6043
Calinski harabasz score is 7.1159
Davies bouldin score is 371.1056


,File,nSample,Hash,Cluster,Has_n/a,nLabel,nCluster,Time(s),Homo.,Sil.,Cal.,Dav.
0,mb_1K,1000.0,tlsh,hac,1.0,56.0,55.0,0.6222,0.4868,-0.5495,10.0111,84.7388
1,mb_1K,1000.0,tlsh,hac-t,1.0,56.0,85.0,0.2077,0.3453,-0.7152,3.6595,55.2025
2,mb_1K,1000.0,tlsh,dbscan,1.0,56.0,71.0,0.9750,0.3336,-0.7217,4.3249,39.4504
3,mb_1K,1000.0,tlsh,kmeans,1.0,56.0,55.0,0.1795,0.4641,-0.6043,7.1159,371.1056


In [10]:
res.labels_

array([28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28, 28,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,
       40, 40, 40, 40, 40, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20,
       20, 20, 20, 20, 20, 20, 20, 20, 20, 34, 34, 34, 34, 34, 34, 34, 34,
       34, 34, 34, 34, 34, 34, 34, 34, 34, 34, 34,  7,  7,  7,  7,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7, 48, 48, 48, 48, 48, 48,
       48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 25, 25, 25, 25, 25, 25, 25,
       25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 38, 38, 38, 38, 38, 38, 38,
       38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 14, 14, 14, 14, 14, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 29, 29, 29, 29, 29, 29,
       29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, 46,
       46, 46, 46, 46, 46

### Output

In [11]:
outfile = path + "/output/" + filename + "_result.csv"
df.to_csv(outfile, index = False)

In [12]:
toc = round(time.perf_counter() - tic,4)
toc

2.3652

# Affinity Propagation

In [13]:
from sklearn.cluster import AffinityPropagation
"""
def runAP(tlist,n):
    tdata = tlist2cdata(tlist) 
    #print(tlist)
    #print(tdata)
    res = AffinityPropagation(random_state=5).fit_predict(tdata) 
    return(res)
"""

'\ndef runAP(tlist,n):\n    tdata = tlist2cdata(tlist) \n    #print(tlist)\n    #print(tdata)\n    res = AffinityPropagation(random_state=5).fit_predict(tdata) \n    return(res)\n'

In [14]:
start = time.perf_counter()

res = runAP(tlist,5)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")

#outfile = path + "/output/" + filename + "_kmean_out.txt"
#outputClusters(outfile, tlist, res.labels_, labelList)

#dict = getResult("tlsh","ap",labelList,res.labels_)
#df = df.append(dict, ignore_index = True)
#df

ValueError: Affinity must be 'precomputed' or 'euclidean'. Got <function sim_affinity at 0x000001F309988430> instead

In [ ]:
res

# Mean Shift

In [ ]:
from sklearn.cluster import MeanShift

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = MeanShift(bandwidth=2).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

In [ ]:
res.labels_

# Spectral Clustering

In [ ]:
from sklearn.cluster import SpectralClustering

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = SpectralClustering(n_clusters=nlable, assign_labels='discretize', 
                         random_state=0).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

In [ ]:
res.labels_

# OPTICS

In [17]:
from sklearn.cluster import OPTICS

start = time.perf_counter()

res = runOPTICS(tlist,2)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

Code ran in 2.178 seconds


C:\Users\user\anaconda3\lib\site-packages\sklearn\cluster\_optics.py:807: RuntimeWarning: divide by zero encountered in true_divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


array([  0,  -1,  -1,  -1, 117,  -1,   7,  -1, 117,  -1,   1,  68,  38,
         7,  -1,  -1,  14,  -1,  -1,  -1,  49, 193, 183,  79,  42,  38,
        -1, 103,  92, 106,  -1,  38,  69,  -1,  69,  36,   7,  94,  -1,
        -1,  -1,  -1,  29,  -1, 169,  74,  74,  36,  33,  71,  97,  97,
       162, 163,  88,  97, 146,  31,  -1,  -1,  31,  -1,  -1, 115,  54,
        54,  49,  -1, 148,  42,  72, 112, 107, 160, 147,  -1,  66,  33,
       160,  -1,  -1, 176,  -1, 174, 174, 175, 175, 179, 180, 108,  -1,
        -1, 113,  -1, 156, 112,  -1,  -1,  -1,  -1,   1, 158, 151,  -1,
        -1,  -1,  -1,  -1, 198,  -1,  -1,  66,  66,  -1, 195, 168,  -1,
       148, 148,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  73,  64,  73,  -1,
        -1,  -1,  64, 186, 186, 185, 186, 186, 185,  33,  -1,  -1, 152,
       152, 152, 152,  50, 152,  49, 147,  49, 111,   2,   2,  -1,  71,
        -1,  -1,  -1,  43,  43, 190, 169,  77,  -1, 111,  -1, 156,  -1,
        47,   0,  79,  -1,  34, 185, 188,  55,  55,   2, 186,  -

In [18]:
res.labels_

AttributeError: 'numpy.ndarray' object has no attribute 'labels_'

# Birch

In [ ]:
from sklearn.cluster import Birch

start = time.perf_counter()

tdata = tlist2cdata(tlist)

res = Birch(n_clusters=nlable).fit(tdata)

end = round(time.perf_counter() - start,4)
print("Code ran in " + str(end) + " seconds")
res

In [ ]:
res.labels_